## UTILIZANDO **GRADIO** PARA VISUALIZAÇÃO DAS CLASSIFICAÇÕES (SINGLE OU MULTI LABEL)

- **Gradio** é uma ferramenta em Python que simplifica a criação e compartilhamento de demos ou aplicativos web para modelos de ML, APIs ou funções, sem necessidade de experiência em JavaScript, CSS ou hospedagem web.
- Neste _notebook_, abordaremos a criação de uma **interface gráfica** básica utilizando o **Gradio** e como executar um modelo de **classificação de objetos** em conjunto com essa interface.

#### 🔎 Verificar se está sendo utilizado CPU ou GPU

In [1]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

#
---

### 1º PASSO: IMPORTAR AS BIBLIOTECAS

In [2]:
import os
import zipfile

import gradio as gr
import numpy as np
import tensorflow as tf

c:\Users\saothalm\AppData\Local\miniconda3\envs\vertex_course\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 
---

### 2º PASSO: CRIAR A INTERFACE RESPONSÁVEL POR EXECUTAR O MODELO

Vamos empregar **variáveis globais** para simplificar o código, porém essa não é a única forma de implementar a função. 
_(Sinta-se à vontade para otimizar o código se desejar)_

In [11]:
# Diretório onde o modelo está armazenado
DATA_PATH = os.path.join(os.pardir, "data", "models", "multi")

# Nome do modelo .tflite a ser utilizado
MODEL_FILE = "t0_raspberry_id_multi.tflite"

# Caminho completo do diretório até o arquivo do modelo
MODEL_PATH = os.path.join(DATA_PATH, MODEL_FILE)

#
---

### 3º PASSO: EXTRAIR AS CATEGORIAS DO MODELO QUE SERÁ EXECUTADO

O código abaixo imprime no _output_ quais categorias existem dentro do modelo carregado na variável global **MODEL_PATH**

In [12]:
with zipfile.ZipFile(MODEL_PATH, "r") as archive:
    for file in archive.namelist():
        LABELS = archive.read(file).decode(encoding="utf-8").split()
        display(LABELS)

['model_b_2018_2_heatsinks',
 'model_b_plus_2017_3heatsinks',
 'model_b_plus_2017_noheatsink']

O código abaixo extrai o arquivo **dict.txt** do modelo e o salva no mesmo diretório

In [13]:
with zipfile.ZipFile(MODEL_PATH, "r") as zip_ref:
    zip_ref.extractall(DATA_PATH)

#
---

### 4º PASSO: CRIAR O INTERPRETADOR QUE USARÁ O MODELO E ALOCAR NA MEMÓRIA

In [14]:
# Criação do interpretador do modelo
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)

# Alocação na memória
interpreter.allocate_tensors()

#
---

### 5º PASSO: CRIAR FUNÇÃO DE PREVISÃO COM O INTERPRETADOR DO MODELO

Esta é a função que irá rodar o modelo usando do interpretador alocado e das variáveis globais

In [15]:
def predict(image):
    global LABELS
    global interpreter

    # Recolher detalhes sobre os parâmetros de entrada e saída do modelo.
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    # Reajustar para o tamanho 224x224
    image = image.resize((224, 224))

    # Imagem será normalizada para escala entre 0-1
    image_arr = np.array(image, dtype=np.float32) / 255.0

    # Quantizar o float32 para uint8
    if input_details["dtype"] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        image_arr = image_arr / input_scale + input_zero_point

    # Expandir a array em 1 dimensão
    image_arr = np.expand_dims(image_arr, axis=0).astype(input_details["dtype"])

    # Jogar para memoria do modelo
    interpreter.set_tensor(input_details["index"], image_arr)

    # Fazer inferencia da image
    interpreter.invoke()

    # Jogar em uma variavel o que tem na saida do modelo
    output = interpreter.get_tensor(output_details["index"])[0]

    display(output)

    # Retorna o valor de uint8 para flot32
    if output_details["dtype"] == np.uint8:
        output_scale, output_zero_point = output_details["quantization"]
        output = output_scale * (output.astype(np.float32) - output_zero_point)

    confidences = {LABELS[i]: float(confidence) for i, confidence in enumerate(output)}

    print(confidences, output.argmax())

    return confidences

#### Exemplo de Parâmetros de ENTRADA e SAÍDA do modelo

In [16]:
input_details = interpreter.get_input_details()[0]
display(input_details)

{'name': 'image',
 'index': 0,
 'shape': array([  1, 224, 224,   3], dtype=int32),
 'shape_signature': array([  1, 224, 224,   3], dtype=int32),
 'dtype': numpy.uint8,
 'quantization': (0.007874015718698502, 128),
 'quantization_parameters': {'scales': array([0.00787402], dtype=float32),
  'zero_points': array([128], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [17]:
output_details = interpreter.get_output_details()[0]
display(output_details)

{'name': 'scores',
 'index': 172,
 'shape': array([1, 3], dtype=int32),
 'shape_signature': array([1, 3], dtype=int32),
 'dtype': numpy.uint8,
 'quantization': (0.00390625, 0),
 'quantization_parameters': {'scales': array([0.00390625], dtype=float32),
  'zero_points': array([0], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

# 
---

### 6º PASSO: CRIAR A INTERFACE COM O GRADIO

Criação da interface que utiliza da função **_predict(image)_**

In [18]:
gr.Interface(
    fn=predict,
    inputs=[gr.Image(type="pil")],
    outputs=[gr.Label()],
).launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


array([111, 105,  51], dtype=uint8)

{'model_b_2018_2_heatsinks': 0.43359375, 'model_b_plus_2017_3heatsinks': 0.41015625, 'model_b_plus_2017_noheatsink': 0.19921875} 0


array([ 97, 100,  84], dtype=uint8)

{'model_b_2018_2_heatsinks': 0.37890625, 'model_b_plus_2017_3heatsinks': 0.390625, 'model_b_plus_2017_noheatsink': 0.328125} 1


array([108, 105,  87], dtype=uint8)

{'model_b_2018_2_heatsinks': 0.421875, 'model_b_plus_2017_3heatsinks': 0.41015625, 'model_b_plus_2017_noheatsink': 0.33984375} 0


array([ 77, 158,  65], dtype=uint8)

{'model_b_2018_2_heatsinks': 0.30078125, 'model_b_plus_2017_3heatsinks': 0.6171875, 'model_b_plus_2017_noheatsink': 0.25390625} 1


####
---

#### 💡VALIDE O MODELO ATRAVÉS DA INTERFACE
- É possível testar o modelo pelo próprio _output_ da célula do notebook
- Outra opção é clicar no link gerado e testá-lo em seu _browser_ de preferência